In [1]:
import pandas as pd
import numpy as np
import matplotlib as ptl

In [2]:
chase_full = pd.read_csv(r"C:\Users\willd\OneDrive\Desktop\Budget\statements\transaction_csv\Chase0818_Activity20221231_20231118_20231119.CSV")
pnc_full = pd.read_csv(r"C:\Users\willd\OneDrive\Desktop\Budget\statements\transaction_csv\PNCaccountActivityExport.csv")

In [3]:
# function to fill ammount column in the pnc dataframe 
def amount_fill(data):
    if '$' in str(data['Withdrawals']):
        amount = '-' + data['Withdrawals']
    else:
        amount = data['Deposits']
    return float(amount.replace('$', '').replace(',', ''))

In [4]:
pnc_full['Amount'] = pnc_full.apply(amount_fill, axis=1)

In [5]:
chase_full = chase_full.rename(columns={"Transaction Date": "Date"})

In [41]:
# add catacories to pnc data frame 
df = pnc_full['Description']
cat_condition = [(df.str.contains('VENMO')), (df.str.contains('SPOTIFY'))]
cat_value = ['VENMO', 'SPOTIFY']
pnc_full['Category'] = np.select(cat_condition, cat_value)

In [42]:
pnc_full

,Date,Description,Withdrawals,Deposits,Category,Balance,Amount
0,2023-11-16,ACH WEB XXXXX9263 CHASE CREDIT CRD EPAY,$361.88,NaN,0,$403.34,-361.88
1,2023-11-15,ONLINE TRANSFER TO XXXXX5748,$800.00,NaN,0,$765.22,-800.00
2,2023-11-15,"ACH CREDIT 07987436 ACTALENT, INC. DIR DEP",NaN,"$1,188.79",0,"$1,565.22",1188.79
3,2023-11-09,ACH WEB XXXXX6175 VENMO PAY...,$320.82,NaN,VENMO,$376.43,-320.82
4,2023-11-09,ACH WEB XXXXX8633 CHASE CREDIT CRD EPAY,$350.00,NaN,0,$697.25,-350.00
5,2023-11-08,ONLINE TRANSFER TO XXXXX5748,$500.00,NaN,0,"$1,047.25",-500.00
6,2023-11-08,"ACH CREDIT 07987436 ACTALENT, INC. DIR DEP",NaN,"$1,188.78",0,"$1,547.25",1188.78
7,2023-11-03,CHECK 150 009542228,"$1,680.00",NaN,0,$358.47,-1680.00
8,2023-11-02,ACH WEB XXXXX5071 CHASE CREDIT CRD EPAY,$150.00,NaN,0,"$2,038.47",-150.00
9,2023-11-01,ONLINE TRANSFER TO XXXXX5748,"$1,000.00",NaN,0,"$2,188.47",-1000.00


In [6]:
import datetime

In [7]:
# Convert date columns to datetime and withdraw/depo to float
chase_full['Date'] = pd.to_datetime(chase_full['Date'])
pnc_full['Date'] = pd.to_datetime(pnc_full['Date'])

In [8]:
# reduce both data frames to relevant columns
chase_reduced = chase_full[["Date", 'Description', "Category", "Amount"]]
pnc_reduced = pnc_full[["Date", 'Description', "Category", "Amount"]]

In [9]:
chase_reduced

,Date,Description,Category,Amount
0,2023-11-15,Payment Thank You-Mobile,NaN,361.88
1,2023-11-13,SONIC DRIVE IN #4415,Food & Drink,-15.19
2,2023-11-10,BARRELS &amp; BOTTLES,Food & Drink,-71.56
3,2023-11-09,KING SOOPERS #0682 FUE,Gas,-55.20
4,2023-11-07,KING SOOPERS #0082,Groceries,-9.87
...,...,...,...,...
277,2023-01-03,TWISTED MINDS,Shopping,-59.59
278,2023-01-04,LinkedIn 7806759426,Bills & Utilities,-43.19
279,2023-01-05,Payment Thank You-Mobile,NaN,142.54
280,2023-01-03,FIRSTENERGY/EZPAYRECUR,Bills & Utilities,-52.17


In [10]:
pnc_reduced

,Date,Description,Category,Amount
0,2023-11-16,ACH WEB XXXXX9263 CHASE CREDIT CRD EPAY,-,-361.88
1,2023-11-15,ONLINE TRANSFER TO XXXXX5748,-,-800.00
2,2023-11-15,"ACH CREDIT 07987436 ACTALENT, INC. DIR DEP",-,1188.79
3,2023-11-09,ACH WEB XXXXX6175 VENMO PAY...,-,-320.82
4,2023-11-09,ACH WEB XXXXX8633 CHASE CREDIT CRD EPAY,-,-350.00
5,2023-11-08,ONLINE TRANSFER TO XXXXX5748,-,-500.00
6,2023-11-08,"ACH CREDIT 07987436 ACTALENT, INC. DIR DEP",-,1188.78
7,2023-11-03,CHECK 150 009542228,-,-1680.00
8,2023-11-02,ACH WEB XXXXX5071 CHASE CREDIT CRD EPAY,-,-150.00
9,2023-11-01,ONLINE TRANSFER TO XXXXX5748,-,-1000.00


In [35]:
# remove CC Payments from both 
pnc_no_cc = pnc_reduced[~pnc_reduced['Description'].str.contains('CHASE CREDIT CRD EPA')]
# remove pnc transfers 
pnc_no_transfer = pnc_no_cc[~pnc_no_cc['Description'].str.contains('ONLINE TRANSFER')]
chase_no_cc = chase_reduced[chase_reduced['Amount'] < 0]

In [37]:
# reduce to september 2023 on 
chase_sept = chase_no_cc[chase_no_cc['Date'] >= '2023-09-01']
pnc_sept = pnc_no_transfer[pnc_no_transfer['Date'] >= '2023-09-01']

In [38]:
pnc_sept

,Date,Description,Category,Amount
2,2023-11-15,"ACH CREDIT 07987436 ACTALENT, INC. DIR DEP",-,1188.79
3,2023-11-09,ACH WEB XXXXX6175 VENMO PAY...,-,-320.82
6,2023-11-08,"ACH CREDIT 07987436 ACTALENT, INC. DIR DEP",-,1188.78
7,2023-11-03,CHECK 150 009542228,-,-1680.00
10,2023-11-01,"ACH CREDIT 07987436 ACTALENT, INC. DIR DEP",-,1188.79
12,2023-10-30,RECURRING DEBIT CARD XXXXX9303 APPLECOMBILL ...,-,-0.99
13,2023-10-27,ACH CREDIT XXXXX5018 VENMO CAS...,-,850.00
15,2023-10-26,RECURRING DEBIT CARD XXXXX9299 SPOTIFY ...,-,-11.87
16,2023-10-25,NON-PNC ATM SURCHARGE REIMBURSEMENT,-,3.00
17,2023-10-25,ATM TRANSACTION FEE REIMBURSEMENT,-,3.00


In [39]:
chase_sept

,Date,Description,Category,Amount
1,2023-11-13,SONIC DRIVE IN #4415,Food & Drink,-15.19
2,2023-11-10,BARRELS &amp; BOTTLES,Food & Drink,-71.56
3,2023-11-09,KING SOOPERS #0682 FUE,Gas,-55.20
4,2023-11-07,KING SOOPERS #0082,Groceries,-9.87
5,2023-11-07,MCDONALD'S F6010,Food & Drink,-12.44
...,...,...,...,...
76,2023-09-04,PARKMOBILE,Travel,-5.45
78,2023-09-03,IKON PASS,Entertainment,-929.00
79,2023-09-02,WM SUPERCENTER #4567,Groceries,-107.95
80,2023-09-02,UBER EATS,Food & Drink,-65.59


In [20]:
# create useful dataframes from PNC
income = pnc_sept[pnc_sept['Description'].str.contains('ACTALENT, INC')]
venmo = pnc_sept[pnc_sept['Description'].str.contains('VENMO')]
rent =   pnc_sept[pnc_sept['Description'].str.contains('CHECK')]
cc_payments = pnc_sept[pnc_sept['Description'].str.contains('CHASE CREDIT')]
rent =   pnc_sept[pnc_sept['Description'].str.contains('CHECK')]

In [21]:
print(len(pnc_sept))
print(len(income))
print(len(venmo))
print(len(rent))
print(len(cc_payments))

51
11
5
2
14


In [17]:
venmo

,Date,Description,Withdrawals,Deposits,Category,Balance
3,2023-11-09,ACH WEB XXXXX6175 VENMO PAY...,$320.82,NaN,-,$376.43
13,2023-10-27,ACH CREDIT XXXXX5018 VENMO CAS...,NaN,$850.00,-,"$2,261.67"
32,2023-10-02,ACH WEB XXXXX9523 VENMO PAY...,$899.92,NaN,-,"$2,329.34"
35,2023-09-27,ACH CREDIT XXXXX5291 VENMO CAS...,NaN,$909.11,-,"$3,514.72"
49,2023-09-05,ACH WEB XXXXX2168 VENMO PAY...,$860.75,NaN,-,$619.97


In [19]:
rent

,Date,Description,Withdrawals,Deposits,Category,Balance
7,2023-11-03,CHECK 150 009542228,"$1,680.00",NaN,-,$358.47
31,2023-10-02,CHECK 148 010880600,"$1,560.00",NaN,-,$769.34
